<a href="https://colab.research.google.com/github/AnIsAsPe/Fundamentos-tecnicos-y-aplicaciones-ML/blob/main/Semana%203/%203_2%20Clasificaci%C3%B3n_de_im%C3%A1genes_c%C3%A1ncer_mediante_programaci%C3%B3n_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de imágenes mediante programación lineal

## Cargar bibliotecas

In [1]:
!pip install --upgrade --user ortools
#https://developers.google.com/optimization/reference/python/linear_solver/pywraplp

Requirement already up-to-date: ortools in /root/.local/lib/python3.6/site-packages (8.1.8487)


In [1]:
import numpy as np   #librería para manejar matrices y operaciones de matrices 
import pandas as pd  #librería para manejar tablas de datos

                                       #Skimage (Scikit-image): librería para procesamiento de imagenes
from skimage import io                 #Modulo para leer una imagen (librería para procesamiento de imagenes)
from skimage.transform import rescale  #Función dentro del modulo transform, para cambiar el tamaño de una imagen 


import matplotlib.pyplot as plt        #Para graficar y visualizar
import seaborn as sns

from ortools.linear_solver import pywraplp

# para especificar que queremos las graficas en linea sin necesidad de usar plt.show()
%matplotlib inline      

## Funciones


#### clasificacion_pl()

In [2]:
#Modificada por mi


def clasificacion_pl(A,B):
  ya = 1
  yb = -1

  d = A.shape[1]
  N_a = A.shape[0]
  N_b = B.shape[0]

  # 1. Declaramos el 'solver'

  solver =  pywraplp.Solver('Clasificador', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

  # 2. Creamos las variables (1 z  por c/imagen de C, 
  #                           1 a por c/imagen de S, y
  #                           1 beta por cada coeficiente + Bo)

  infinito = solver.infinity()
  za = [solver.NumVar(0, infinito, 'ya'+str(i)) for i in range(N_a)] 
  zb = [solver.NumVar(0, infinito, 'ya'+str(i)) for i in range(N_b)] 
  beta = [solver.NumVar(-infinito, infinito,'B'+str(i)) for i in range(d+1)] 

  print('cantidad de variables =', solver.NumVariables())

  # 3. Definimos las restricciones: 

  for i in range(N_a):
    solver.Add(0 >=  -1* za[i] + 1 - ya * solver.Sum((((beta[j] * A[i][j])+ beta[d]) for j in range(d))))
  for i in range(N_b):
    solver.Add(0 >=  -1* zb[i] + 1 - yb * solver.Sum((((beta[j] * B[i][j])+ beta[d]) for j in range(d))))

  print('Cantidad de restricciones =', solver.NumConstraints())

  # 4. Definimos la función objetivo
  Agap = solver.Sum(za[i] for i in range(N_a))
  Bgap = solver.Sum(zb[i] for i in range(N_b))

  solver.Minimize( Agap + Bgap)

  # 5. Solucionamos el problema
  status = solver.Solve()
  if status  == pywraplp.Solver.OPTIMAL:
    print('Solución:')
    print('Valor objetivo =', solver.Objective().Value())

    Bs = []
    for bt in beta:
      b = bt.solution_value()
      Bs.append(b)
  else:
    raise Exception('The problem does not have an optimal solution.')
  return np.array(Bs)

  

## Lectura y exploración de datos

In [3]:
df = pd.read_csv(('/content/drive/MyDrive/Datos/img_cancer_26x26pixeles_con_etiqueta.csv'))
print(df.shape)
df.head()

(5063, 677)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,clase
0,0.083223,0.083223,0.083223,0.083225,0.083405,0.086613,0.094121,0.095760,0.095939,0.095679,0.095800,0.096059,0.095816,0.097095,0.099579,0.104371,0.121509,0.151148,0.166485,0.142229,0.100965,0.084538,0.083237,0.083223,0.083223,0.083223,0.083223,0.083223,0.083240,0.084179,0.092702,0.113767,0.125615,0.128518,0.128971,0.129441,0.130263,0.130819,0.130473,0.130028,...,0.188656,0.188316,0.192373,0.195840,0.194437,0.196929,0.193218,0.175780,0.128152,0.089850,0.083352,0.083223,0.083223,0.083223,0.083223,0.083223,0.083259,0.085485,0.109178,0.145949,0.147400,0.147373,0.153215,0.136331,0.128650,0.127448,0.124526,0.123044,0.123262,0.126940,0.126231,0.126898,0.121795,0.099420,0.084719,0.083252,0.083223,0.083223,0.083223,0
1,0.083223,0.083223,0.083223,0.083241,0.084892,0.112470,0.192035,0.219843,0.187999,0.180500,0.186738,0.186549,0.189735,0.188186,0.190577,0.193022,0.192217,0.186904,0.186196,0.171629,0.112510,0.085044,0.083240,0.083223,0.083223,0.083223,0.083223,0.083223,0.083305,0.089308,0.159444,0.323082,0.435133,0.479868,0.425607,0.406373,0.416591,0.418254,0.416060,0.420284,...,0.350737,0.328330,0.313045,0.315160,0.312431,0.316695,0.318328,0.278893,0.166274,0.092502,0.083354,0.083223,0.083223,0.083223,0.083223,0.083223,0.083301,0.089128,0.159247,0.262119,0.234541,0.216824,0.201565,0.194039,0.190225,0.190590,0.191628,0.185784,0.175968,0.180713,0.180095,0.184120,0.171356,0.119763,0.086424,0.083276,0.083223,0.083223,0.083223,0
2,0.083223,0.083223,0.083223,0.083240,0.084891,0.108390,0.155804,0.162704,0.179060,0.174836,0.160696,0.163860,0.181359,0.174879,0.160358,0.162727,0.175161,0.189507,0.215191,0.226223,0.136367,0.086611,0.083256,0.083223,0.083223,0.083223,0.083223,0.083223,0.083272,0.087915,0.154537,0.309475,0.352005,0.328503,0.358213,0.373566,0.346785,0.339945,0.390837,0.392600,...,0.332288,0.331581,0.343050,0.345295,0.330323,0.323152,0.320105,0.279885,0.169279,0.092595,0.083367,0.083223,0.083223,0.083223,0.083223,0.083223,0.083344,0.091139,0.171438,0.296291,0.282717,0.251271,0.217253,0.197555,0.193422,0.184207,0.180436,0.170264,0.177799,0.194378,0.195019,0.178956,0.169096,0.118785,0.086534,0.083280,0.083223,0.083223,0.083223,0
3,0.083223,0.083223,0.083223,0.083241,0.084968,0.108683,0.163460,0.192673,0.178441,0.165327,0.172069,0.186495,0.178956,0.167983,0.162857,0.169700,0.184739,0.207286,0.224597,0.247589,0.140882,0.086850,0.083257,0.083223,0.083223,0.083223,0.083223,0.083223,0.083273,0.088753,0.156897,0.300279,0.347244,0.385530,0.391457,0.352362,0.352138,0.404086,0.410804,0.362560,...,0.364815,0.359175,0.341390,0.352469,0.337021,0.346255,0.342502,0.302029,0.177949,0.093565,0.083382,0.083223,0.083223,0.083223,0.083223,0.083223,0.083307,0.089709,0.165956,0.278964,0.248794,0.222636,0.206214,0.199162,0.194352,0.190506,0.195654,0.206230,0.195324,0.191298,0.182867,0.184448,0.180449,0.122997,0.086934,0.083285,0.083223,0.083223,0.083223,0
4,0.083223,0.083223,0.083223,0.083243,0.085150,0.125502,0.223968,0.188510,0.181894,0.181329,0.172335,0.169007,0.173832,0.184832,0.188771,0.204627,0.220360,0.238141,0.270100,0.229637,0.128614,0.086290,0.083253,0.083223,0.083223,0.083223,0.083223,0.083223,0.083290,0.089593,0.165005,0.366662,0.548632,0.433108,0.428869,0.439226,0.422466,0.382484,0.364961,0.389379,...,0.356768,0.350259,0.345512,0.349427,0.360185,0.365872,0.349878,0.325673,0.188289,0.094558,0.083407,0.083223,0.083223,0.083223,0.083223,0.083223,0.083289,0.087291,0.130107,0.201024,0.221397,0.227356,0.207675,0.190300,0.199108,0.204295,0.192606,0.190277,0.187772,0.186235,0.187462,0.198557,0.191787,0.125617,0.087338,0.083290,0.083223,0.083223,0.083223,0


In [4]:
df['clase'].value_counts()

1    3594
0    1469
Name: clase, dtype: int64

In [5]:
df.loc[df['clase']==0,'clase']=-1
df['clase'].value_counts()

 1    3594
-1    1469
Name: clase, dtype: int64

In [6]:
#El método GroupBy de Pandas separa un data frame en varios data frames
por_clase =df.groupby('clase')

#trabajaremos con la misma cantidad de datos de una clase que de otra
S = por_clase.get_group(-1)
C = por_clase.get_group(1).sample(n = len(S), random_state=3)
datos = pd.concat([S, C])

datos['clase'].value_counts()

 1    1469
-1    1469
Name: clase, dtype: int64

In [7]:
# Separar en conjunto de entrenamiento y conjunto de prueba
prueba = datos.sample(frac=.3, random_state=3)

entrenamiento = pd.concat([datos,prueba]).drop_duplicates(keep=False).reset_index(drop=True)
prueba = prueba.reset_index(drop=True)

for dat in [prueba, entrenamiento]:
  print(dat.shape)
  print(dat['clase'].value_counts(),'\n')

(881, 677)
 1    453
-1    428
Name: clase, dtype: int64 

(2057, 677)
-1    1041
 1    1016
Name: clase, dtype: int64 



In [8]:
  #El método GroupBy de Pandas separa un data frame en varios data frames
  por_clase =entrenamiento.groupby('clase')

  #separamos los dataframes de acuerdo a la clase 
  #eliminamos la etiqueta
  #lo convertimos de regreso a una matriz de numpy

  S = por_clase.get_group(-1).iloc[:,:-1].to_numpy()  #conjunto de imágenes de tejido cáncerígeno
  C = por_clase.get_group(1).iloc[:,:-1].to_numpy()  #conjunto de imágenes de tejido sano

print(type(C), type(S), '\n')
print(C.shape,  S.shape )

<class 'numpy.ndarray'> <class 'numpy.ndarray'> 

(1016, 676) (1041, 676)


# Clasificación de imágenes con programación lineal

## Desarrollo de la solución

In [188]:
A = C  
B = S 
ya = 1
yb = -1

d = A.shape[1]
N_a = A.shape[0]
N_b = B.shape[0]

# 1. Declaramos el 'solver'

solver =  pywraplp.Solver('Clasificador', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

# 2. Creamos las variables (1 z  por c/imagen de C, 
#                           1 a por c/imagen de S, y
#                           1 beta por cada coeficiente + Bo)

infinito = solver.infinity()
za = [solver.NumVar(0, infinito, 'ya'+str(i)) for i in range(N_a)] 
zb = [solver.NumVar(0, infinito, 'ya'+str(i)) for i in range(N_b)] 
beta = [solver.NumVar(-infinito, infinito,'B'+str(i)) for i in range(d+1)] 

print('cantidad de variables =', solver.NumVariables())

# 3. Definimos las restricciones: 

for i in range(N_a):
  solver.Add(0 >=  -1* za[i] + 1 - ya * solver.Sum((((beta[j] * A[i][j])+ beta[d]) for j in range(d))))
for i in range(N_b):
  solver.Add(0 >=  -1* zb[i] + 1 - yb * solver.Sum((((beta[j] * B[i][j])+ beta[d]) for j in range(d))))

print('Cantidad de restricciones =', solver.NumConstraints())

cantidad de variables = 2141
Cantidad de restricciones = 1464


In [189]:
  
# 5. Definimos la función objetivo
Agap = solver.Sum(za[i] for i in range(N_a))
Bgap = solver.Sum(zb[i] for i in range(N_b))

solver.Minimize(Agap + Bgap)

In [190]:
%%time
status = solver.Solve()
status

CPU times: user 22.1 s, sys: 53.8 ms, total: 22.2 s
Wall time: 22.1 s


In [191]:
status

0

In [192]:
pywraplp.Solver.OPTIMAL

0

In [193]:
if status == pywraplp.Solver.OPTIMAL:
  print('Solution:')
  print('Objective value =', solver.Objective().Value())
  Bs = []
  for bt in beta:
    b = bt.solution_value( )
    Bs.append(b)
else:
  print('The problem does not have an optimal solution.')

Solution:
Objective value = 0.0


In [63]:
Bs = []
for bt in beta:
  b = bt.solution_value( )
  Bs.append(b)


In [202]:
betas = np.array(Bs)
len(betas)  #una beta por cada dimensión + B_0


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -4.23193602e+02,  7.22806047e+01, -5.91883068e+01,
        5.23715439e+01, -4.49846382e+01,  1.09717882e+01, -6.95146523e+00,
        4.00236329e+00, -5.94508980e+00,  1.75131536e+01, -2.18193191e+01,
        3.39291752e+01, -2.30728038e+01,  2.84692680e+00,  7.02790979e+01,
       -1.07639254e+02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  3.11370952e+03, -2.13249820e+02,  6.16643071e+01,
        3.10807939e+00,  5.64543298e+00,  0.00000000e+00, -7.75507155e+00,
        3.11082583e+01, -4.54714908e+01,  4.58661079e+01, -1.88852099e+01,
        0.00000000e+00,  4.44227170e+00,  1.10477071e+01, -2.71410381e+01,
        2.95183833e+01, -5.35290147e+01,  3.07674521e+01, -4.26743691e+01,
        7.53314150e+02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

## Todo junto

In [9]:

%time betas = clasificacion_pl(C,S)

cantidad de variables = 2734
Cantidad de restricciones = 2057
Solución:
Valor objetivo = 0.0
CPU times: user 1min 24s, sys: 381 ms, total: 1min 24s
Wall time: 1min 24s


In [12]:

len(betas)

677

In [13]:
betas[660:670]

array([ -24.40361868,  -17.70807669,    8.92183345,  -13.94207934,
        -10.43483874,   -3.40342777,   71.39822103, -103.79356172,
         84.45223116,   29.90739932])

## Predicción y evaluación del modelo

* La imágen se clasifica como tejido cancerígeno (-1) si:

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\langle x_{i},~ \beta~ \rangle+\beta_{0}<0$
* La imágen se clasifica como tejido sano (1) si :
$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\langle x_{i},~ \beta~ \rangle+\beta_{0}>0$

In [15]:
entrenamiento['prediccion'] = np.where((entrenamiento.iloc[:,0:675].dot(betas[0:675])+betas[-1])<0,-1,1)
pd.crosstab(entrenamiento['clase'], entrenamiento['prediccion'])

prediccion,-1,1
clase,,
-1,1041,0
1,0,1016


In [22]:
prueba['prediccion'] = np.where(((prueba.iloc[:,0:675].dot(betas[0:675]))+betas[-1])<0,-1,1)
matriz_confusion = pd.crosstab(prueba['clase'], prueba['prediccion'])
matriz_confusion

prediccion,-1,1
clase,,
-1,311,117
1,164,289
